In [36]:
import pandas as pd
import json

In [37]:
folder = '.\\input_files\\'
file = '341883132_bay-park-hospital_standardcharges.json'

In [38]:
with open(folder + file, 'r') as f:
    jd = json.load(f)
    jd = jd[0]['item']

In [39]:
df = pd.DataFrame(jd)

In [40]:
df = df.assign(Associated_Codes=df['Associated_Codes'].str.split(',')).explode('Associated_Codes')

In [41]:
df.rename(columns={
    'payer': 'payer_name',
    'Associated_Codes': 'code',
    'iobSelection': 'setting',
    'Payer_Allowed_Amount': 'standard_charge',
}, inplace=True)

In [42]:
df_payer = df.copy()
df_payer = df_payer[['payer_name', 'description', 'code', 'standard_charge', 'setting']]
df_payer['payer_category'] = 'payer'

In [43]:
df_rates = df.copy()
df_rates = df_rates[['description', 'code', 'setting', 'Gross_Charge', 'Cash_Discount', 'Deidentified_Min_Allowed', 'DeIdentified_Max_Allowed']]

In [44]:
cols = df_rates.columns.tolist()
id_vars = cols[:3]
value_vars = cols[3:]

df_rates = pd.melt(df_rates, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')

In [45]:
mapping = {
    'Gross_Charge': 'gross',
    'Cash_Discount': 'cash',
    'Deidentified_Min_Allowed': 'min',
    'DeIdentified_Max_Allowed':'max',
    }

df_rates['payer_category'] = df_rates['payer_name'].map(mapping)

In [46]:
df = pd.concat([df_payer, df_rates])

In [47]:
df.reset_index(drop=True, inplace=True)

In [48]:
df['setting'] = df['setting'].str.lower()

In [49]:
df.loc[df['code'].str.match(r'^[A-Z][0-9]{4}$|^[0-9]{5}$|^[0-9]{4}[A-Z]$'), 'hcpcs_cpt'] = df['code']
df.loc[df['code'].str.match(r'^[0-9]{3}$'), 'ms_drg'] = df['code']
df.loc[df['code'].str.len() == 4, 'rev_code'] = df['code']

In [52]:
df.loc[df['standard_charge'] == 'N/A', 'standard_charge'] = pd.NA

In [55]:
df.dropna(subset='standard_charge', inplace=True)

In [57]:
id_mapping = {
 '344428256_toledo-hospital_standardcharges copy.json': '360074',
 '341883132_bay-park-hospital_standardcharges.json': '360259',
 '344446484_defiance-regional-hospital_standardcharges.json': '361328',
 '344428256_toledo-hospital_standardcharges.json': '360068',
 '340898745_fostoria-community-hospital_standardcharges.json': '361318',
 '344430849_memorial-hospital_standardcharges.json': '360156',
 '382796005_charles-and-virginia-hickman-hospital_standardcharges.json': '230005',
 '386108110_coldwater-regional-hospital_standardcharges.json': '230022',
 '381984289_monroe-regional-hospital_standardcharges.json': '230099'}

hosp_id = id_mapping[file]

df['hospital_id'] = hosp_id

out_file = hosp_id + file.split('_')[1] + '.csv'

out_folder = '.\\output_files\\'

df.to_csv(out_folder + out_file, index=False)

,payer_name,description,code,standard_charge,setting,payer_category,hcpcs_cpt,ms_drg,rev_code
141,MEDICAL MUTUAL Commercial,Lab Test - Identification of DNA from 22 Patho...,0097U,397.8000,outpatient,payer,0097U,NaN,NaN
142,PARAMOUNT Commercial,Lab Test - Identification of DNA from 22 Patho...,0097U,269.2800,outpatient,payer,0097U,NaN,NaN
148,UNITED HEALTHCARE Commercial,Lab Test - Identification of DNA from 22 Patho...,0097U,402.7600,outpatient,payer,0097U,NaN,NaN
313,PARAMOUNT MEDICARE ADVANTAGE,Fine Needle Aspiration of Cells for Biopsy Usi...,10005,559.7200,outpatient,payer,10005,NaN,NaN
334,AETNA BETTER HEALTH MANAGED CARE,Incision and Drainage of Superficial Soft Tiss...,10061,252.9300,outpatient,payer,10061,NaN,NaN
338,ANTHEM Commercial,Incision and Drainage of Superficial Soft Tiss...,10061,1324.8600,outpatient,payer,10061,NaN,NaN
350,PARAMOUNT MANAGED MEDICAID,Incision and Drainage of Superficial Soft Tiss...,10061,320.2800,outpatient,payer,10061,NaN,NaN
7323,PRIORITY HEALTH MEDICARE ADVANTAGE,CAT Scan of Head or Brain Without Contrast,70450,103.5400,outpatient,payer,70450,NaN,NaN
9305,UNITED HEALTHCARE MANAGED MEDICAID,CAT Scan of Abdomen and Pelvis With Contrast,74177,195.6400,outpatient,payer,74177,NaN,NaN
9677,PRIORITY HEALTH MANAGED MEDICAID,External Ultrasound of Pregnant Uterus in 1st ...,76801,56.3800,outpatient,payer,76801,NaN,NaN


In [63]:
df[df.duplicated(subset=['payer_name','description','code','setting','payer_category','hcpcs_cpt','ms_drg','rev_code'], keep=False)]

,payer_name,description,code,standard_charge,setting,payer_category,hcpcs_cpt,ms_drg,rev_code
5460,ANTHEM Commercial,Replacement Of Knee Joint,469,17793.8200,inpatient,payer,NaN,469,NaN
5514,ANTHEM Commercial,Replacement Of Knee Joint,469,20197.7800,inpatient,payer,NaN,469,NaN
20072,Gross_Charge,Lab Test - Identification of DNA from 22 Patho...,0097U,816.0000,outpatient,gross,0097U,NaN,NaN
20073,Gross_Charge,Lab Test - Identification of DNA from 22 Patho...,0097U,816.0000,outpatient,gross,0097U,NaN,NaN
20074,Gross_Charge,Lab Test - Identification of DNA from 22 Patho...,0097U,816.0000,outpatient,gross,0097U,NaN,NaN
...,...,...,...,...,...,...,...,...,...
99674,DeIdentified_Max_Allowed,Lab Test - COVID-19 Test Panel By Non-CDC Lab ...,U0004,194.3600,outpatient,max,U0004,NaN,NaN
99675,DeIdentified_Max_Allowed,Lab Test - COVID-19 Test Panel By Non-CDC Lab ...,U0004,194.3600,outpatient,max,U0004,NaN,NaN
99676,DeIdentified_Max_Allowed,Lab Test - COVID-19 Test Panel By Non-CDC Lab ...,U0004,194.3600,outpatient,max,U0004,NaN,NaN
99677,DeIdentified_Max_Allowed,Lab Test - COVID-19 Test Panel By Non-CDC Lab ...,U0004,194.3600,outpatient,max,U0004,NaN,NaN


In [59]:
file.split('_')

['341883132', 'bay-park-hospital', 'standardcharges.json']